## Chatbot

This chatbot can keep context

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="gemma2-9b-it")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001F5481CF0D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001F5482095A0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, My name is Upamanyu and I'm an MTech CSE student")])

AIMessage(content="Hi Upamanyu!\n\nThat's great to know! MTech in Computer Science Engineering is a fascinating field.  \n\nWhat area of CSE are you specializing in? Are you working on any interesting projects right now?\n\nI'm happy to chat with you about anything related to technology, academics, or even just general interests.  Let me know what's on your mind!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 26, 'total_tokens': 108, 'completion_time': 0.149090909, 'prompt_time': 0.002095529, 'queue_time': 0.26487378, 'total_time': 0.151186438}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--563fdf00-4d3f-4588-a28a-16384e854570-0', usage_metadata={'input_tokens': 26, 'output_tokens': 82, 'total_tokens': 108})

In [4]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi, My name is Upamanyu and I'm an MTech CSE student"),
    AIMessage(content="Hello Upamanyu!\n\nIt's nice to meet you. That's great! MTech in Computer Science and Engineering is a challenging and rewarding field. What area of CSE are you specializing in?\n\nI'm happy to chat with you about anything related to technology, programming, or your studies if you'd like.\n"),
    HumanMessage(content="Hey What's my name and what do I do?")
])

AIMessage(content="As a large language model, I don't have memory of past conversations. \n\nYou introduced yourself as **Upamanyu**, an **MTech CSE student**. 😊 \n\n\nLet me know if you have any other questions.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 117, 'total_tokens': 167, 'completion_time': 0.090909091, 'prompt_time': 0.00565026, 'queue_time': 0.25818028, 'total_time': 0.096559351}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a66520e2-53d8-49b5-a3ff-9e17c829d325-0', usage_metadata={'input_tokens': 117, 'output_tokens': 50, 'total_tokens': 167})

### Message history

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}
def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable": {"session_id": "chat1"}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Upamanyu and I'm an MTech CSE student")],
    config=config
)

In [8]:
response.content

"Hi Upamanyu,\n\nIt's nice to meet you!\n\nBeing an MTech CSE student is exciting. What area of computer science are you specializing in?  Do you have any specific projects or interests you're working on?\n\nI'm happy to chat about anything related to computer science, or just general topics if you'd like.  😊\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config
)

AIMessage(content='Your name is Upamanyu.  \n\nYou told me at the beginning of our conversation! 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 118, 'total_tokens': 142, 'completion_time': 0.043636364, 'prompt_time': 0.00570955, 'queue_time': 0.263926508, 'total_time': 0.049345914}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d9980591-bea8-4930-b4bd-92ea76549570-0', usage_metadata={'input_tokens': 118, 'output_tokens': 24, 'total_tokens': 142})

#### Changing the config and seesion id

In [10]:
config1 = {"configurable": {"session_id": "chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to use it!\n"

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey my name is George")],
    config=config1
)
response.content

"Hi George! It's nice to meet you. What can I do for you today? 😊  \n\n"

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config1
)
response.content

"Your name is George, that's what you told me!  \n\nIs there anything else I can help you with, George?\n"

### Prompt Templates

In [14]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [15]:
chain.invoke({"messages": [HumanMessage(content="Hi, My name is Upamanyu")]})

AIMessage(content="Hello Upamanyu, it's nice to meet you!\n\nWhat can I do for you today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 32, 'total_tokens': 60, 'completion_time': 0.050909091, 'prompt_time': 0.003313179, 'queue_time': 0.454246029, 'total_time': 0.05422227}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--0d33d12c-c9f6-4ebb-b477-26814c6a4ea3-0', usage_metadata={'input_tokens': 32, 'output_tokens': 28, 'total_tokens': 60})

In [16]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history=get_session_history)

In [17]:
config = {"configurable": {"session_id": "chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Upamanyu")],
    config=config
)

response.content

"Hi Upamanyu! It's nice to meet you.\n\nWhat can I do for you today? 😊  \n"

#### Adding more complexity

In [18]:

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer the questions to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model

In [22]:
response = chain.invoke({
    "messages": [HumanMessage(content="Hi, My name is Upamanyu")], 
    "language": "Bengali"
    })

response.content

'নমস্কার, Upamanyu! 😊  আমি তোমার সাহায্যকারী।  কী প্রশ্ন তুমি জানতে চাও?  আমি যা জানি তা সব শেয়ার করবো। \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history

In [23]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="messages"
)

In [24]:
config = {"configurable": {"session_id": "chat4"}}
response = with_message_history.invoke(
    {
        "messages" : [HumanMessage(content="Hi, I'm Upamanyu"),],
        "language" : "Bengali"
    },
    config=config
)
response.content

'নমস্কার, Upamanyu!  😊 \n\nআপনি কিভাবে আছেন? আমি আপনার সাহায্যে তোমাকে সর্বোত্তম ভাবে সাহায্য করার জন্য প্রস্তুত।  \n\nআপনি কোন প্রশ্ন করতে চান?\n'

In [25]:
response = with_message_history.invoke(
    {
        "messages" : [HumanMessage(content="What's my name?"),],
        "language" : "Bengali"
    },
    config=config
)
response.content

'আপনার নাম Upamanyu. 😊  \n'

### Manage the Conversation history

'trim_messages' helper to resuce how many messages we're sending to the model

In [28]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=50,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")
]

trimmer.invoke(messages)


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [29]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough


chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What ice cream do I like?")],
    "language": "English"
    }
)

response.content

"As a large language model, I don't have access to your personal information like your favorite ice cream flavor.  \n\nWhat's *your* favorite ice cream? 😊\n"

In [30]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What math problem did I ask for?")],
    "language": "English"
    }
)

response.content

'You asked me "what\'s 2 + 2". 😊  \n\n\n\nLet me know if you want to try another one! \n'

### Lat's wrap this in the message history

In [35]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_session_history,
    input_messages_key="messages"
)

config={"configurable": {"session_id": "chat5"}}

In [37]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What's my name?")],
        "language": "English"
    },
    config=config
)

response.content

"You haven't told me your name!  What would you like me to call you? 😊  \n"